# Surface

## Index

- [Definition](#Definition)
- [List of defined Surfaces](#List-of-defined-Surfaces)
- [Example : Plotting a Cylinder](#Example-:-Plotting-a-Cylinder)
- [Creating new surface](#Creating-new-surface)
    - [Example : MySurface](#Example-:-My-Surface)
    - [Surface Integrity Check](#Surface-Consistancy-Check)
    - Partially defined Surface
- [Evaluation](#Evaluation)
- [Transformations](#Transformations)
    - Translation
    - Rotation
    - Translation and Rotation
- Surface area calculation
- Poles
    - Example of the Sphere
- Numerical Surface
- Comments about precision

## Definition

A *surface* represented by the class `gsurface.surface.surface.Surface`, is defined using 2 parameters designated by $u$ and $v$. See complete definition on [Wikipedia](https://en.wikipedia.org/wiki/Surface_(mathematics)).

$$
\begin{align*}
\mathbf{S} : \mathbb{R}^2 &\to\mathbb{R}^3 \\
u, v &\mapsto \mathbf{S}(u, v)
\end{align*}
$$

## List of defined Surfaces

Following basic surfaces are already completly defined in the library (`gsurface.surface` sub-package):

* Plan : `.plan.Plan` with parameters $a$, $b$, $c$, $d$ from plan definition equation $ax + by + cz = d$
* Cylinder : `.cylinder.Cylinder` with parameter $R$
* Sphere : `.sphere.Sphere` with parameter $R$
* Tore : `.tore.Tore` with parameters $r$ and $R$


* Paraboloids with parameters $a$ and $b$
    * EllipticParaboloid : `.paraboloid.EllipticParaboloid`
    * HyperbolicParaboloid: `.paraboloid.HyperbolicParaboloid`
    

* Catenoid : `.catenoid.Catenoid` with parameter $a$
* ConicalCorner : `.conicalcorner.ConicalCorner` with parameters $k$ and $a$

* EggBox : `.eggbox.Eggbox` with parameters $R$, $a$ and $b$

## Example : Plotting a Cylinder

A cylinder of radius $r$ can be defined as follow :

$$
\mathbf{Cyl}(u, v) = \begin{cases}
R cos(u) \\
R sin(u) \\
v
\end{cases}
$$

The `gsurface.surface.cylinder.Cylinder` class implements this surface. We can plot this surface using *mlab* and *mayavi_plot_surface* function.

In [1]:
from gsurface.surface import Cylinder
from gsurface.plotter import mayavi_plot_surface

cyl = Cylinder(R = 1.0)  # creating object

cyl.setlims(v_ll = 0, v_ul = 10)  # updating v bounds to [0, 10]

mesh = cyl.mesh(50, 50)  # creating u,v-mesh

smesh = cyl.build_surface(*mesh)  # generating surface points for this u,v-mesh

mayavi_plot_surface(smesh)  # plotting

**Symbols** :

- `cyl` is a *Surface:Cylinder object*
- `mesh` is as *2-dim np.ndarray* (50x50)
- `smesh` is as *3-dim np.ndarray* (50x50x3)

![mayavi_cylinder_ui.png](./pics/mayavi_cylinder_ui.png)

## Creating new surface

The `gsurface.surface` sub-package allows you to simply define your own parametric surfaces.

In [7]:
import numpy as np

from gsurface.surface import Surface, SJH

### Example : My Surface

For example Here we want to define the following parametric surface :

$$S (u, v)=\left[\begin{matrix}u - \cos{\left(v \right)}\\v \sin{\left(u \right)} + v\\u \sin{\left(u - v \right)} - v\end{matrix}\right]$$

In order to build our surface class, we need to find the Jacobian and the Hessians matrices :

$$J = \begin{bmatrix}
\frac{\partial S_x}{\partial u} & \frac{\partial S_x}{\partial v} \\
\frac{\partial S_y}{\partial u} & \frac{\partial S_y}{\partial v} \\
\frac{\partial S_z}{\partial u} & \frac{\partial S_z}{\partial v} \\
\end{bmatrix} =
\begin{bmatrix}
1 & \sin(v) \\
v \cos(u) & \sin(u) + 1 \\
u \cos(u - v) + \sin(u - v) & u \cos(u - v) - 1
\end{bmatrix}
$$

$$H_x = \begin{bmatrix}
\frac{\partial^2 S_x}{\partial^2 u} & \frac{\partial^2 S_x}{\partial u \partial v} \\
\frac{\partial^2 S_x}{\partial u \partial v} & \frac{\partial^2 S_x}{\partial^2 v}
\end{bmatrix} =
\begin{bmatrix}
0 & 0 \\
0 & \cos(v)
\end{bmatrix}$$

$$H_y = \begin{bmatrix}
\frac{\partial^2 S_y}{\partial^2 u} & \frac{\partial^2 S_y}{\partial u \partial v} \\
\frac{\partial^2 S_y}{\partial u \partial v} & \frac{\partial^2 S_y}{\partial^2 v}
\end{bmatrix} =
\begin{bmatrix}
v \sin(u) & \cos(u) \\
\cos(u) & 0
\end{bmatrix}$$

$$H_z = \begin{bmatrix}
\frac{\partial^2 S_z}{\partial^2 u} & \frac{\partial^2 S_z}{\partial u \partial v} \\
\frac{\partial^2 S_z}{\partial u \partial v} & \frac{\partial^2 S_z}{\partial^2 v}
\end{bmatrix} =
\begin{bmatrix}
- u \sin(u - v) + 2 \cos(u - v) & u \sin(u - v) - \cos(u - v) \\
u \sin(u - v) - \cos(u - v) & - u \sin(u - v)
\end{bmatrix}$$

Finally we can choose parameters limits, for example :
$u, v \in [-2, 2]^2$

In [22]:
class MySurface(Surface):
    def __init__(self):
        # initializing base surface and setting limits
        super().__init__(
            plimits=np.array([
                [-2.0, 2.0],
                [-2.0, 2.0]
            ])
        )

    # this function must return a tuple of S, J, and Hxyz matrices
    # by using the buildMetric function we can simplify the definition
    # not filled values are set to 0.0
    def _definition(self, u: float, v: float) -> SJH:        
        return self.buildMetric(
            # S
            x=u - np.cos(v),
            y=v*np.sin(u) + v,
            z=np.sin(u - v)*u - v,
            
            # J
            dux=1,
            duy=v*np.cos(u),
            duz=u*np.cos(u - v) + np.sin(u - v),

            dvx=np.sin(v),
            dvy=np.sin(u) + 1,
            dvz=-u*np.cos(u - v) - 1,

            # Hx
            dvvx = np.cos(v),

            # Hy
            duuy=-v*np.sin(u),
            duvy=np.cos(u),

            # Hz
            duuz=-u*np.sin(u - v) + 2*np.cos(u - v),
            duvz=u*np.sin(u - v) - np.cos(u - v),
            dvvz=-u*np.sin(u - v),
        )
        

### Surface Consistancy Check

The `gsurface.surface.diff` file offer a tool in order to check that your newly defined surface is consistant, in the sense that the Jacobian and Hessians matrices are well calculated.

The `gsurface.surface.surface.Surface` `check` and `check_verbose` methods check that each first order and seconds order derivates are well calulated in the surface model. These methods are based on the `gsurface.surface.diff.get_diff_surface_errors` function.

The `gsurface.surface.diff.get_diff_surface_errors` function need a mesh of $u, v \in M_u \otimes M_v$ parameters where to evaluate each derivatives. ($M_v$ and $M_v$ are finite sets of real numbers). On each position defined by the mesh, for each differentiated values the function compare the defined derivative to the numerical approximation of the derivative.

If one of the *error* ($\varepsilon$) calculated is greater than the *tolerance value* (by default $10^{-7}$), the `check` methods return false.


#### Example

Here we check our previously well-defined `MySurface` surface, but if make a willful mistake on the derivative of a value in the previous definition and we perfom the test again we get an error.

In [23]:
surface = MySurface()

result = surface.check_verbose(20, 20)

print(f"Surface model valid ? {result}")

=== Checking surface definition ===
U 20 points from -2.0 to 2.0
V 20 points from -2.0 to 2.0
Tolerance : 1.0E-07
index[0 : xi] max err = 4.2E-08 [True]
index[1 : yi] max err = 5.2E-08 [True]
index[2 : zi] max err = 6.5E-08 [True]
index[3 : duxi] max err = 0.0E+00 [True]
index[4 : dvxi] max err = 1.1E-08 [True]
index[5 : duyi] max err = 2.8E-08 [True]
index[6 : dvyi] max err = 1.4E-08 [True]
index[7 : duzi] max err = 5.3E-08 [True]
index[8 : dvzi] max err = 6.9E-08 [True]
Surface model valid ? True


In [28]:
class BadSurface(MySurface):
    def _definition(self, u: float, v: float) -> SJH:
        S, J, H = super()._definition(u, v)
        
        # we introduce a sign error on DSy/Dv
        J[1, 1] = -J[1, 1]
        
        return S, J, H
    
# we perform the consistancy check again
bad_surface = BadSurface()

result = bad_surface.check_verbose(20, 20)

print(f"Surface model valid ? {result}")  # check fails

=== Checking surface definition ===
U 20 points from -2.0 to 2.0
V 20 points from -2.0 to 2.0
Tolerance : 1.0E-07
index[0 : xi] max err = 4.2E-08 [True]
index[1 : yi] max err = 4.0E+00 [False]
index[2 : zi] max err = 6.5E-08 [True]
index[3 : duxi] max err = 0.0E+00 [True]
index[4 : dvxi] max err = 1.1E-08 [True]
index[5 : duyi] max err = 2.8E-08 [True]
index[6 : dvyi] max err = 2.0E+00 [False]
index[7 : duzi] max err = 5.3E-08 [True]
index[8 : dvzi] max err = 6.9E-08 [True]
Surface model valid ? False


#### Comments
- Importance of scale in definitions, change tolerance consequently.
- [Schwartz theorem](https://en.wikipedia.org/wiki/Symmetry_of_second_derivatives) says that $\frac{\partial^2 f}{\partial u \partial v} = \frac{\partial^2 f}{\partial v \partial u}$. We will always talk about `duvx` for $\frac{\partial^2 S_x}{\partial u \partial v} = \frac{\partial^2 S_x}{\partial v \partial u}$, same for `duvy` and `duvz`.

## Evaluation

The point associated to the parametric position $(u, v)$ is $\mathbf{S}(u, v)$.

For the previous cylinder the point associated to the parametric position $\left(\frac{pi}{2}, 1\right)$ is $\mathbf{Cyl}\left(\frac{pi}{2}, 1\right) = \begin{bmatrix} 0 \\ R \\ 1 \end{bmatrix}$

In [4]:
import numpy as np

S, J, H = cyl.eval(np.pi / 2, 1)

print(S)

[6.123234e-17 1.000000e+00 1.000000e+00]


the `eval` function return 3 values $\mathbf{S}$ the position of the point on the surface (dimension $3$), $\mathbf{J}$ the [Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) of $S$ (dimension $3\times2$) and [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix) of $S_x$, $S_y$ and $S_z$ : $\mathbf{H} = \begin{bmatrix} \mathbf{H_x} & \mathbf{H_y} & \mathbf{H_z} \end{bmatrix}$ (dimension $(2\times2)$ each), all evaluated on the parametric position $(u, v)$.

These 3 values are gathered in the tuple `gsurface.types.SJH` of the library.

## Transformations

### Translation

### Rotation

### Rotation and Translation

Check surface consistancy

Create model

In [4]:
from gsurface import SurfaceGuidedMassSystem, Tyi, build_s0
from gsurface.forces import ViscousFriction, SpringForce

model = SurfaceGuidedMassSystem(
    surface=surface,
    s0=build_s0(u0=2.5, du0=-8.0, dv0=5.0),
    m=1.0,
    forces=[
        SpringForce(10.0),
        ViscousFriction(1.0)
    ]
)

In [5]:
from gsurface.plotter import mayavi_plot_surfaces, SurfacePlot

surface  = NewSurface().multlims(1.5)

surface_mesh = surface.build_surface(*surface.mesh(50, 50))

time = np.linspace(0, 10, 1000)

data = model.solve(time)

solutions = model.solutions(data, time)

# animate does not work with jupyter notebooks
mayavi_plot_surfaces([
    SurfacePlot(surface_mesh, trajectory=solutions[Tyi], animate=False)
])

![New Surface Trajectory](../render/NewSurfaceTraj.png)